## Installing pdfplumber to deal with pdfs

In [1]:
# Uncomment the line below if required
# pip install pdfplumber

## Importing the required libraries

In [2]:
from collections import namedtuple
import re
import pdfplumber
import pandas as pd
import numpy as np

### Format of the dataframe columns

In [3]:
Line = namedtuple('Line','home_university college_id college_name branch_id branch_name gopens_or_gopenh rank')

#### Using regex to identify College name,College id,Branch name and Branch id

In [4]:
college_re = re.compile(r'^(\d{4}) \- (.*)')
branch_re = re.compile(r'^(\d{9}) \- (.*)')

### Mining of data across all pages of pdf

In [26]:
data=[]
with pdfplumber.open(r"C:\Users\Mohit Singh\Desktop\college_cutoffs.pdf") as pdf:
    pages = pdf.pages
    for page in pages:
        text = page.extract_text(x_tolerance = 2, y_tolerance =0)
        rtext= text.split('\n')
        i=0
        
        while(i<len(rtext)):
            g=""
            hom =""
            coll = college_re.search(rtext[i])
            rank = 0
            branch = branch_re.search(rtext[i])
            if coll:
                college_id, college_name = coll.group(1), coll.group(2)
            if branch:
                branch_id , branch_name = branch.group(1), branch.group(2)
                if branch_name == "Civil Engineering":
                    while(rtext[i]!="Status:"):
                        i=i+1
                    i=i+1
                    if(len(rtext[i].split("  "))>1):
                        hom = rtext[i].split("  ")[1][1:]
                    else:
                        hom = rtext[i].split("  ")[0][1:]
                    
                    
                    while(i<len(rtext) and rtext[i].split(" ")[0] != "GOPENS" and rtext[i].split()[0]!="GOPENH"):
                        i=i+1
                    if(i<len(rtext)):
                        g= rtext[i].split(" ")[0]
                    while(i<len(rtext) and rtext[i] != "S"):
                        i=i+1
                    i=i+1
                    if(i<len(rtext)):
                        rank = (int)(rtext[i].split(" ")[0])

                    data.append(Line(hom, college_id, college_name, branch_id, branch_name, g , rank))
            i+=1

data[0:5]

[Line(home_university='Sant Gadge Baba Amravati University', college_id='1002', college_name='Government College of Engineering, Amravati', branch_id='100219110', branch_name='Civil Engineering', gopens_or_gopenh='GOPENS', rank=9698),
 Line(home_university='Dr. Babasaheb Ambedkar Technological University,Lonere', college_id='1012', college_name='Government Engineering College, Yavatmal', branch_id='101219110', branch_name='Civil Engineering', gopens_or_gopenh='GOPENH', rank=20224),
 Line(home_university='Sant Gadge Baba Amravati University', college_id='1105', college_name='Prof. Ram Meghe Institute of Technology & Research, Amravati', branch_id='110519110', branch_name='Civil Engineering', gopens_or_gopenh='GOPENH', rank=32149),
 Line(home_university='Sant Gadge Baba Amravati University', college_id='1105', college_name='Prof. Ram Meghe Institute of Technology & Research, Amravati', branch_id='110519120', branch_name='Civil Engineering', gopens_or_gopenh='GOPENH', rank=38173),
 Line(h

### Lookup at a single page

In [22]:
import pdfplumber
data1= []
with pdfplumber.open(r"C:\Users\Mohit Singh\Desktop\college_cutoffs.pdf") as pdf:
    first_page = pdf.pages[16]
    text = first_page.extract_text(x_tolerance = 2, y_tolerance =0)
    rtext= text.split('\n')
    i=0
    while(i<len(rtext)):
        g=""
        rank=""
        hom =""
        coll = college_re.search(rtext[i])
        branch = branch_re.search(rtext[i])
        if coll:
            college_id, college_name = coll.group(1), coll.group(2)
        if branch:
            branch_id , branch_name = branch.group(1), branch.group(2)
            if branch_name == "Civil Engineering":
                while(rtext[i]!="Status:"):
                    i=i+1
                i=i+1
                hom = rtext[i].split("  ")[0][1:]
                while(i<len(rtext) and rtext[i].split(" ")[0] != "GOPENS" and rtext[i].split()[0]!="GOPENH"):
                    i=i+1
                if(i<len(rtext)):
                    g= rtext[i].split(" ")[0]
                while(i<len(rtext) and rtext[i] != "S"):
                    i=i+1
                i=i+1
                if(i<len(rtext)):
                    rank = (int)(rtext[i].split(" ")[0])

                data1.append(Line(hom, college_id, college_name, branch_id, branch_name, g , rank))
        i+=1

data1


[Line(home_university='Sant Gadge Baba Amravati University', college_id='1105', college_name='Prof. Ram Meghe Institute of Technology & Research, Amravati', branch_id='110519110', branch_name='Civil Engineering', gopens_or_gopenh='GOPENH', rank=32149)]

### Building of the Dataframe 

In [29]:
df = pd.DataFrame(data)
df.head()

,home_university,college_id,college_name,branch_id,branch_name,gopens_or_gopenh,rank
0,Sant Gadge Baba Amravati University,1002,"Government College of Engineering, Amravati",100219110,Civil Engineering,GOPENS,9698
1,Dr. Babasaheb Ambedkar Technological Universit...,1012,"Government Engineering College, Yavatmal",101219110,Civil Engineering,GOPENH,20224
2,Sant Gadge Baba Amravati University,1105,Prof. Ram Meghe Institute of Technology & Rese...,110519110,Civil Engineering,GOPENH,32149
3,Sant Gadge Baba Amravati University,1105,Prof. Ram Meghe Institute of Technology & Rese...,110519120,Civil Engineering,GOPENH,38173
4,Sant Gadge Baba Amravati University,1107,P. R. Pote (Patil) Education & Welfare Trust's...,110719110,Civil Engineering,GOPENH,43898


### Extracting the dataframe according to 'Savitribai Phule Pune University

In [28]:
df1 = df.loc[df['home_university']=='Savitribai Phule Pune University']

df1.head()

### Sorting of the Dataframe to get top 10 colleges as desired in given task

In [31]:
df1.sort_values('rank').head(10)

,home_university,college_id,college_name,branch_id,branch_name,gopens_or_gopenh,rank
168,Savitribai Phule Pune University,6006,"College of Engineering, Pune",600619110,Civil Engineering,GOPENS,1255
166,Savitribai Phule Pune University,6004,"Government College of Engineering & Research, ...",600419110,Civil Engineering,GOPENH,12456
180,Savitribai Phule Pune University,6175,"Pimpri Chinchwad Education Trust, Pimpri Chinc...",617519110,Civil Engineering,GOPENH,12971
210,Savitribai Phule Pune University,6289,B.R.A.C.T's Vishwakarma Institute of Informati...,628919110,Civil Engineering,GOPENS,14178
175,Savitribai Phule Pune University,6146,"MIT Academy of Engineering,Alandi, Pune",614619110,Civil Engineering,GOPENS,16173
131,Savitribai Phule Pune University,5121,K. K. Wagh Institute of Engineering Education ...,512119110,Civil Engineering,GOPENH,22937
223,Savitribai Phule Pune University,6325,Alard Charitable Trust's Alard College of Eng...,632519110,Civil Engineering,GOPENH,23543
161,Savitribai Phule Pune University,5401,Jawahar Education Society's Institute of Techn...,540119110,Civil Engineering,GOPENH,27370
206,Savitribai Phule Pune University,6278,All India Shri Shivaji Memorial Society's Coll...,627819110,Civil Engineering,GOPENH,28635
151,Savitribai Phule Pune University,5184,Amruta Vaishnavi Education & Welfare Trust's S...,518419110,Civil Engineering,GOPENH,32066


** Here is the result of the given task as per your requirement**